In [84]:
import requests

API_KEY = "76fc507d-66dd-44b8-b5eb-c350f65fa1c7"
url = "https://api.harvardartmuseums.org/classification"

params = {
    "apikey": API_KEY,
    "size": 100
}

response = requests.get(url, params)
response   

<Response [200]>

In [85]:
# STEP 2: CLASSIFICATIONS WHERE OBJECT COUNT >= 2500

# convert response to JSON format
data = response.json()

# loop through each record and filter
print("Classifications with object count >= 2500:\n")

for record in data.get("records", []):
    if record.get("objectcount", 0) >= 2500:
        print(record.get("name"), "-", record.get("objectcount"))

Classifications with object count >= 2500:

Accessories (non-art) - 2862
Photographs - 85836
Drawings - 33961
Prints - 72253
Paintings - 6863
Sculpture - 6643
Coins - 20270
Vessels - 6223
Textile Arts - 3446
Archival Material - 15737
Fragments - 5218
Manuscripts - 5001
Seals - 5899
Straus Materials - 5128


In [86]:
# STEP 3: FETCH OBJECT RECORDS FOR A CHOSEN CLASSIFICATION

# choose a classification from the previous output 
classification = "Vessels"

# API endpoint for object data
url = "https://api.harvardartmuseums.org/object"

# page size and starting page
page = 1
page_size = 100

# empty list to store all records
all_records = []

# run until you collect about 2500 records (25 pages × 100)
while len(all_records) < 2500:
    params = {
        "apikey": API_KEY,
        "classification": classification,
        "size": page_size,
        "page": page
    }

    response = requests.get(url, params=params)
    if response.status_code != 200:
        print("Request failed at page", page)
        break

    data = response.json()
    records = data.get("records", [])

    # stop if no more data
    if not records:
        print("No more records found.")
        break

    # add new records to list
    all_records.extend(records)
    print(f"Fetched page {page} | Total records so far: {len(all_records)}")

    page += 1

# final record count
print("\nTotal records fetched:", len(all_records))

Fetched page 1 | Total records so far: 100
Fetched page 2 | Total records so far: 200
Fetched page 3 | Total records so far: 300
Fetched page 4 | Total records so far: 400
Fetched page 5 | Total records so far: 500
Fetched page 6 | Total records so far: 600
Fetched page 7 | Total records so far: 700
Fetched page 8 | Total records so far: 800
Fetched page 9 | Total records so far: 900
Fetched page 10 | Total records so far: 1000
Fetched page 11 | Total records so far: 1100
Fetched page 12 | Total records so far: 1200
Fetched page 13 | Total records so far: 1300
Fetched page 14 | Total records so far: 1400
Fetched page 15 | Total records so far: 1500
Fetched page 16 | Total records so far: 1600
Fetched page 17 | Total records so far: 1700
Fetched page 18 | Total records so far: 1800
Fetched page 19 | Total records so far: 1900
Fetched page 20 | Total records so far: 2000
Fetched page 21 | Total records so far: 2100
Fetched page 22 | Total records so far: 2200
Fetched page 23 | Total reco

In [87]:
# STEP 4: SPLIT THE DATA INTO 3 PARTS — METADATA, MEDIA, COLORS

metadata = []
media = []
colors = []

for i in all_records:
    # METADATA section
    metadata.append(dict(
        id=i.get("id"),
        title=i.get("title"),
        culture=i.get("culture"),
        period=i.get("period"),
        century=i.get("century"),
        medium=i.get("medium"),
        dimensions=i.get("dimensions"),
        description=i.get("description"),
        department=i.get("department"),
        classification=i.get("classification"),
        accessionyear=i.get("accessionyear"),
        accessionmethod=i.get("accessionmethod")
    ))

    # MEDIA section
    media.append(dict(
        objectid=i.get("id"),
        imagecount=i.get("imagecount"),
        mediacount=i.get("mediacount"),
        colorcount=i.get("colorcount"),
        rank=i.get("rank"),
        datebegin=i.get("datebegin"),
        dateend=i.get("dateend")
    ))

    # COLORS section (list inside each record)
    if i.get("colors"):
        for j in i["colors"]:
            colors.append(dict(
                objectid=i.get("id"),
                color=j.get("color"),
                spectrum=j.get("spectrum"),
                hue=j.get("hue"),
                percent=j.get("percent"),
                css3=j.get("css3")
            ))

print("Metadata records collected:", len(metadata))
print("Media records collected:", len(media))
print("Color records collected:", len(colors))


Metadata records collected: 2500
Media records collected: 2500
Color records collected: 22051


In [15]:
!python -m pip install pymysql


In [92]:
import pymysql
my_db=pymysql.connect(
host ="localhost",
    user="root",
    password=""
    )
print("connection sucessfull.")

my_cursor = my_db.cursor()

# Database creation
my_cursor.execute("CREATE DATABASE IF NOT EXISTS harvard")

print(" Database 'harvard' created or already exists.")


connection sucessfull.
 Database 'harvard' created or already exists.


In [93]:
my_db = pymysql.connect(
    host="localhost",
    user="root",
    password="",
    database="harvard"
)

my_cursor = my_db.cursor()
print("Connected to 'harvard' database.")



Connected to 'harvard' database.


 Create Table: artifact_metadata

In [94]:


my_cursor.execute("""
CREATE TABLE IF NOT EXISTS artifact_metadata (
    id INTEGER PRIMARY KEY,
    title TEXT,
    culture TEXT,
    period TEXT,
    century TEXT,
    medium TEXT,
    dimensions TEXT,
    description TEXT,
    department TEXT,
    classification TEXT,
    accessionyear INTEGER,
    accessionmethod TEXT
)
""")

print("Table 'artifact_metadata' created successfully.")

Table 'artifact_metadata' created successfully.


 Create Table: artifact_media :

In [95]:


my_cursor.execute("""
CREATE TABLE IF NOT EXISTS artifact_media (
    objectid INT PRIMARY KEY,
    imagecount INT,
    mediacount INT,
    colorcount INT,
    rank INT,
    datebegin INT,
    dateend INT,
    FOREIGN KEY (objectid) REFERENCES artifact_metadata(id)
)
""")

print(" Table 'artifact_media' created successfully.")

 Table 'artifact_media' created successfully.


 Create Table: artifact_colors 


In [96]:
 

my_cursor.execute("""
CREATE TABLE IF NOT EXISTS artifact_colors (
    objectid INTEGER NOT NULL,
    color TEXT,
    spectrum TEXT,
    hue TEXT,
    percent REAL,
    css3 TEXT,
    FOREIGN KEY (objectid) REFERENCES artifact_metadata(id),
    PRIMARY KEY (objectid, color(50))
)
""")

print(" Table 'artifact_colors' created successfully.")

 Table 'artifact_colors' created successfully.


In [97]:
# fetched Api data:
import requests

API_KEY = "76fc507d-66dd-44b8-b5eb-c350f65fa1c7"
url = "https://api.harvardartmuseums.org/object"

metadata = []
media = []
colors = []

page = 1
while len(metadata) < 2500:
    params = {
        "apikey": API_KEY,
        "size": 100,
        "page": page
    }

    response = requests.get(url, params=params)
    data = response.json()

    for i in data['records']:
        metadata.append(dict(
            id=i.get('id'),
            title=i.get('title'),
            culture=i.get('culture'),
            period=i.get('period'),
            century=i.get('century'),
            medium=i.get('medium'),
            dimensions=i.get('dimensions'),
            description=i.get('description'),
            department=i.get('department'),
            classification=i.get('classification'),
            accessionyear=i.get('accessionyear'),
            accessionmethod=i.get('accessionmethod')
        ))

        media.append(dict(
            objectid=i.get('id'),
            imagecount=i.get('imagecount'),
            mediacount=i.get('mediacount'),
            colorcount=i.get('colorcount'),
            rank=i.get('rank'),
            datebegin=i.get('datebegin'),
            dateend=i.get('dateend')
        ))

        color_details = i.get('colors')
        if color_details:
            for j in color_details:
                colors.append(dict(
                    objectid=i.get('id'),
                    color=j.get('color'),
                    spectrum=j.get('spectrum'),
                    hue=j.get('hue'),
                    percent=j.get('percent'),
                    css3=j.get('css3')
                ))

        if len(metadata) >= 2500:
            break

    print("Fetched records:", len(metadata))
    page += 1

print("Total metadata records:", len(metadata))
print("Total media records:", len(media))
print("Total color records:", len(colors))

Fetched records: 100
Fetched records: 200
Fetched records: 300
Fetched records: 400
Fetched records: 500
Fetched records: 600
Fetched records: 700
Fetched records: 800
Fetched records: 900
Fetched records: 1000
Fetched records: 1100
Fetched records: 1200
Fetched records: 1300
Fetched records: 1400
Fetched records: 1500
Fetched records: 1600
Fetched records: 1700
Fetched records: 1800
Fetched records: 1900
Fetched records: 2000
Fetched records: 2100
Fetched records: 2200
Fetched records: 2300
Fetched records: 2400
Fetched records: 2500
Total metadata records: 2500
Total media records: 2500
Total color records: 17010


In [98]:
inserted_count = 0

for record in metadata:
    insert_query = """
    INSERT IGNORE INTO artifact_metadata (
        id, title, culture, period, century, medium, dimensions,
        description, department, classification, accessionyear, accessionmethod
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    values = (
        record.get("id"), record.get("title"), record.get("culture"), record.get("period"),
        record.get("century"), record.get("medium"), record.get("dimensions"),
        record.get("description"), record.get("department"), record.get("classification"),
        record.get("accessionyear"), record.get("accessionmethod")
    )

    try:
        my_cursor.execute(insert_query, values)
        inserted_count += 1
    except Exception as e:
        print("Skipped:", e)

my_db.commit()
print("Inserted into artifact_metadata:", inserted_count)


Inserted into artifact_metadata: 2500


In [99]:
#Data Insert in Table 2: artifact_media
inserted_count = 0

for record in media:
    insert_query = """
    INSERT IGNORE INTO artifact_media (
        objectid, imagecount, mediacount, colorcount, rank, datebegin, dateend
    ) VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    values = (
        record.get("objectid"), record.get("imagecount", 0), record.get("mediacount", 0),
        record.get("colorcount", 0), record.get("rank", 0),
        record.get("datebegin", 0), record.get("dateend", 0)
    )

    try:
        my_cursor.execute(insert_query, values)
        inserted_count += 1
    except Exception as e:
        print("Skipped:", e)

my_db.commit()
print("Inserted into artifact_media:", inserted_count)



Inserted into artifact_media: 2500


In [100]:
#Data Insert in Table 3: artifact_colors
inserted_count = 0

for record in colors:
    insert_query = """
    INSERT IGNORE INTO artifact_colors (
        objectid, color, spectrum, hue, percent, css3
    ) VALUES (%s, %s, %s, %s, %s, %s)
    """
    values = (
        record.get("objectid"), record.get("color"), record.get("spectrum"),
        record.get("hue"), record.get("percent"), record.get("css3")
    )

    try:
        my_cursor.execute(insert_query, values)
        inserted_count += 1
    except Exception as e:
        print("Skipped:", e)

my_db.commit()
print("Inserted into artifact_colors:", inserted_count)


Inserted into artifact_colors: 17010


 SQL Queries

In [102]:
query = """
SELECT title
FROM artifact_metadata
WHERE century = '11th century'
  AND culture = 'Byzantine';
"""
my_cursor.execute(query)
results = my_cursor.fetchall()
for row in results:
    print(row[0])  


Seal of Kosmas, Bishop of Proussa
Seal of Sergios chartoularios and kommerkiarios of Presthlavitza (XI c.)
Seal of (John) protoproedros (of the protosynkelloi and metropolitan) of Side (XI c.)
Seal of Nikephoros patrikios and strategos of the Kibyrraiotai (XIth c.)


In [103]:
#2. What are the unique cultures represented in the artifacts?
query = """
SELECT DISTINCT culture
FROM artifact_metadata
WHERE culture IS NOT NULL
ORDER BY culture;
"""

my_cursor.execute(query)
cultures = [row[0] for row in my_cursor.fetchall()]
print(cultures)

['American', 'American?', 'Anatolian', 'Arab', 'Austrian', 'Belgian', 'Brazilian', 'British', 'British, English', 'British, Scottish', 'British?', 'Byzantine', 'Canadian', 'Central Asian?', 'Chinese', 'Chinese?', 'Coptic', 'Cycladic', 'Danish', 'Dutch', 'Dutch?', 'East Asian', 'Egyptian', 'Etruscan', 'European', 'European?', 'Flemish', 'French', 'French?', 'German', 'German?', 'Graeco-Roman', 'Greek', 'Hellenistic', 'Indian', 'Islamic', 'Italian', 'Italian?', 'Japanese', 'Korean', 'Mexican', 'Minoan', 'Moroccan', 'Mughal', 'Mycenaean', 'Near Eastern', 'Netherlandish', 'Parthian', 'Persian', 'Roman', 'Roman Imperial', 'Roman Provincial', 'Roman Republican', 'Russian', 'Siculo-Punic', 'Spanish', 'Sumerian', 'Swedish', 'Swiss', 'Syrian', 'Turkish', 'Unidentified culture', 'Uzbek']


In [106]:
#3. List all artifacts from the Archaic Period.

query = """
SELECT id, title, culture, century, medium, classification
FROM artifact_metadata
WHERE period LIKE '%Archaic%';
"""
my_cursor.execute(query)
results = my_cursor.fetchall()

if results:
    for row in results:
        print(row)
else:
    print("No artifacts found for Archaic Period")


(4618, 'Obol of Himera, Sicily', 'Greek', '5th century BCE', 'Silver', 'Coins')
(4867, 'Basin Handle with Lion and Snake Heads', 'Greek', '6th century BCE', 'Bronze', 'Vessels')
(71082, 'Didrachm of Selinous (Sicily)', 'Greek', '6th century BCE', 'Silver', 'Coins')
(71145, 'Didrachm of Selinous (Sicily)', 'Greek', '6th century BCE', 'Silver', 'Coins')
(71147, 'Didrachm of Selinous (Sicily)', 'Greek', '6th century BCE', 'Silver', 'Coins')


In [107]:
#4. List artifact titles ordered by accession year in descending order
query = """
SELECT title, accessionyear
FROM artifact_metadata
WHERE accessionyear IS NOT NULL
ORDER BY accessionyear DESC;
"""

my_cursor.execute(query)
results = my_cursor.fetchall()

for row in results:
    print(row)

('Standing Draped Man Carrying a Large Bundle on His Shoulder', 2024)
('Allegory of the Triumph of the Netherlands over Spain', 2021)
('Bust-length Portrait of a Young Woman, Facing Left', 2018)
('Saints Prosdocimus, Justina, Daniel, and Anthony of Padua in Glory with a View of the Prato della Valle, Padua', 2018)
('Globular, Wide-Mouthed Jar with Two Loop Handles and Pale Lavender Splash', 2016)
('Large Circular Bowl with Five Large Russet Splashes against a Ground of Russet Flecks', 2016)
('Broad-Shouldered, "Hanging Jar" with Vertical Lip, Wide Mouth, and Landscape Décor', 2016)
('Globular Jar with Rounded Bottom and Flaring Lip', 2016)
('Small Jade Sculpture in the Form of a Recumbent Ram', 2016)
('Small Water Coupe in the Form of a Spotted Frog', 2016)
('"Eye" Bead with Aqua and White Swirls', 2016)
('Small Tea Bowl with Decoration of Moon, Clouds, and Blossoming Plum', 2016)
('Medicine Buddha (Chinese, Yaoshi Rulai) Seated on a Lotus Pedestal and Holding a Small Medicine Jar in H

In [108]:
#5. How many artifacts are there per department?
query = """
SELECT department, COUNT(*) AS artifact_count
FROM artifact_metadata
WHERE department IS NOT NULL
GROUP BY department
ORDER BY artifact_count DESC;
"""

my_cursor.execute(query)
for row in my_cursor.fetchall():
    print(row)

('Department of Photographs', 1355)
('Department of Drawings', 1227)
('Department of Prints', 817)
('Department of Ancient and Byzantine Art & Numismatics', 562)
('Straus Center for Conservation and Technical Studies', 526)
('Busch-Reisinger Museum', 224)
('Department of Asian Art', 143)
('Department of Islamic & Later Indian Art', 72)
('Department of Modern & Contemporary Art', 40)
('Department of Paintings, Sculpture & Decorative Arts', 26)
('Harvard University Portrait Collection', 3)
('Department of American Paintings, Sculpture & Decorative Arts', 3)
('Archives', 2)


SQL OURIES FOR artifact_media

In [109]:
#6. Which artifacts have more than 1 image?
query = """
SELECT m.objectid, d.title, m.imagecount
FROM artifact_media AS m
JOIN artifact_metadata AS d ON m.objectid = d.id
WHERE m.imagecount > 1
ORDER BY m.imagecount DESC;
"""

my_cursor.execute(query)
for row in my_cursor.fetchall():
    print(row)

(69421, 'Volume 4: The Antiquities of Rome, IV', 66)
(59857, 'Volume 6: Collections of Prints', 60)
(6262, 'Charity, Organizations: United States. Massachusetts. Boston. Publicity for Social Work. Annual Reports: A Sound Investment: A Statement. 1924: North Bennet St. Industrial School', 52)
(6875, 'Charity, Organizations: United States. Massachusetts. Boston. Publicity for Social Work. Annual Reports: Only Yesterday I was a Boy', 51)
(59886, 'Volume 5: Various Prints', 39)
(4968, "Charity, Organizations: United States. Massachusetts. Boston. Publicity for Social Work: Booklets: Winter and Spring: General Education Department Young Women's Christian Association Boston, Massachusetts", 36)
(6176, "Charity, Organizations: United States. Massachusetts. Boston. Publicity for Social Work. Annual Reports: Full Measure: The Greater Service of Boston's Y.W.C.A.", 36)
(5017, 'Charity, Organizations: United States. Massachusetts. Boston. Publicity for Social Work: Booklets: Our Citizens: North B

In [110]:
#7. What is the average rank of all artifacts?
query = """
SELECT AVG(rank) AS average_rank
FROM artifact_media
WHERE rank IS NOT NULL;
"""

my_cursor.execute(query)
avg_rank = my_cursor.fetchone()[0]
print("Average Rank of Artifacts:", avg_rank)

Average Rank of Artifacts: 133053.5556


In [111]:
#8. Which artifacts have a higher colorcount than mediacount?
query = """
SELECT m.objectid, d.title, m.colorcount, m.mediacount
FROM artifact_media AS m
JOIN artifact_metadata AS d ON m.objectid = d.id
WHERE m.colorcount > m.mediacount;
"""

my_cursor.execute(query)
for row in my_cursor.fetchall():
    print(row)

(1412, '[Buildings, Germany (Siemensstadt?)]', 3, 0)
(1413, '[Buildings, Stockholm, 1936]', 8, 0)
(1414, '[Street scene, Berlin]', 7, 0)
(1415, '[Street scene in Berlin]', 6, 0)
(1416, '[Shellhaus, Berlin]', 7, 0)
(1417, '[Street scene, Berlin]', 5, 0)
(1418, '[Clothes blowing in the wind, Berlin]', 4, 0)
(1419, '[Building reflected in water, Deep, Baltic Coast]', 6, 0)
(1423, 'Portrait of a Woman, after a Drawing in the Uffizi then attributed to Leonardo da Vinci', 6, 0)
(1424, 'The Parable of the Unjust Husbandmen', 7, 0)
(1425, 'The Virgin and Joseph at the Inn', 8, 0)
(1427, 'The Guards of the Rose Garden', 10, 0)
(1428, 'Battle Scene (painting, recto), folio fragment from a Bhagavata Purana series', 8, 0)
(1429, 'What a Glorious Land!', 7, 0)
(1430, 'Two Equestrian Figures, One Male with a Tall, Elaborately Embellished Hat, One Female with Hair in a Topknot, Both with Pointed Boots, and Hands Positioned to Hold the Reins of their Standing, Saddled Horses', 10, 0)
(1431, 'Statement

In [112]:
#9. List all artifacts created between 1500 and 1600
query = """
SELECT m.objectid, d.title, m.datebegin, m.dateend
FROM artifact_media AS m
JOIN artifact_metadata AS d ON m.objectid = d.id
WHERE m.datebegin >= 1500 AND m.dateend <= 1600;
"""

my_cursor.execute(query)
for row in my_cursor.fetchall():
    print(row)

(1424, 'The Parable of the Unjust Husbandmen', 1585, 1585)
(1425, 'The Virgin and Joseph at the Inn', 1585, 1585)
(1428, 'Battle Scene (painting, recto), folio fragment from a Bhagavata Purana series', 1535, 1545)
(4605, 'Twelve Scenes from the Life of Christ (Emblemata Evangelica) plus title page', 1585, 1585)
(4606, 'The Parable of the Rich Fool', 1585, 1585)
(4607, 'The Plucking of Ears of Corn on Sabbath Day', 1585, 1585)
(4608, 'The Parable of the Barren Fig Tree', 1585, 1585)
(4620, 'The Parable of the Kingdom of Heaven', 1585, 1585)
(4621, 'Titlepage', 1585, 1585)
(4622, 'Christ and the Woman of Samaria', 1585, 1585)
(4623, 'The Good Shepherd', 1585, 1585)
(4624, 'Selection of 16th century prints', 1535, 1577)
(4633, 'The Angel Telling Joseph to Flee and the Flight into Egypt', 1585, 1585)
(4634, 'Jesus Calling the Apostles Among the Fishermen', 1585, 1585)
(4635, 'The Parable of the Sower', 1585, 1585)
(4636, 'The Parable of the Two Sons and the Unjust Husbandmen', 1585, 1585)


In [113]:
#10.How many artifacts have no media files?
query = """
SELECT COUNT(*) AS artifacts_without_media
FROM artifact_media
WHERE mediacount = 0 OR mediacount IS NULL;
"""

my_cursor.execute(query)
count = my_cursor.fetchone()[0]
print("Artifacts without media files:", count)

Artifacts without media files: 5000


QUERIES FOR artifact_colors Table:

In [114]:
#11. What are all the distinct hues used in the dataset?
query = """
SELECT DISTINCT hue
FROM artifact_colors
WHERE hue IS NOT NULL
ORDER BY hue;
"""

my_cursor.execute(query)
hues = [row[0] for row in my_cursor.fetchall()]
print("Distinct Hues:", hues)

Distinct Hues: ['Black', 'Blue', 'Brown', 'Green', 'Grey', 'Orange', 'Red', 'Violet', 'White', 'Yellow']


In [115]:
#12.What are the top 5 most used colors by frequency?
query = """
SELECT color, COUNT(*) AS frequency
FROM artifact_colors
WHERE color IS NOT NULL
GROUP BY color
ORDER BY frequency DESC
LIMIT 5;
"""

my_cursor.execute(query)
top_colors = my_cursor.fetchall()
for color, freq in top_colors:
    print(color, "-", freq)

#969696 - 998
#7d7d7d - 995
#646464 - 852
#c8c8c8 - 794
#afafaf - 764


In [116]:
#13.What is the average coverage percentage for each hue?
query = """
SELECT hue, AVG(percent) AS average_coverage
FROM artifact_colors
WHERE hue IS NOT NULL
GROUP BY hue
ORDER BY average_coverage DESC;
"""

my_cursor.execute(query)
for row in my_cursor.fetchall():
    print(row)

('Black', 0.19689442398564866)
('Yellow', 0.16459173156403054)
('Green', 0.1611653888348463)
('Orange', 0.15613334523532754)
('Grey', 0.1446368185309742)
('Brown', 0.11080604475602979)
('White', 0.10613466868843023)
('Red', 0.06665090029082066)
('Blue', 0.04568481610363174)
('Violet', 0.042391512570535785)


In [118]:
#14.List all colors used for a given artifact ID.
artifact_id = 12345

query = f"""
SELECT color, hue, percent, css3
FROM artifact_colors
WHERE objectid = '{artifact_id}';
"""

my_cursor.execute(query)
results = my_cursor.fetchall()

if results:
    for row in results:
        print(row)
else:
    print(f"No colors found for artifact ID {artifact_id}")


No colors found for artifact ID 12345


In [119]:
#15.What is the total number of color entries in the dataset?
query = "SELECT COUNT(*) AS total_color_entries FROM artifact_colors;"
my_cursor.execute(query)
count = my_cursor.fetchone()[0]
print("Total number of color entries:", count)

Total number of color entries: 17010


JOINS BASED QUERIES:

In [120]:
#16.List artifact titles and hues for all artifacts belonging to the Byzantine culture.
query = """
SELECT m.title, c.hue
FROM artifact_metadata AS m
JOIN artifact_colors AS c
    ON m.id = c.objectid
WHERE m.culture = 'Byzantine'
  AND c.hue IS NOT NULL
ORDER BY m.title;
"""

my_cursor.execute(query)
for row in my_cursor.fetchall():
    print(row)

('Saint Andrew and Scenes from his Life', 'Brown')
('Saint Andrew and Scenes from his Life', 'Brown')
('Saint Andrew and Scenes from his Life', 'Brown')
('Saint Andrew and Scenes from his Life', 'Brown')
('Saint Andrew and Scenes from his Life', 'Yellow')
('Saint Andrew and Scenes from his Life', 'Brown')
('Saint Andrew and Scenes from his Life', 'Yellow')
('Saint Andrew and Scenes from his Life', 'Brown')
('Saint Andrew and Scenes from his Life', 'Green')
('Saint Andrew and Scenes from his Life', 'Orange')


In [121]:
#17.List each artifact title with its associated hues.
query = """
SELECT m.title, c.hue
FROM artifact_metadata AS m
JOIN artifact_colors AS c
    ON m.id = c.objectid
WHERE c.hue IS NOT NULL
ORDER BY m.title;
"""

my_cursor.execute(query)
for row in my_cursor.fetchall():
    print(row)

('"And Gus Taubs boy over in the tank plan."', 'Green')
('"And Gus Taubs boy over in the tank plan."', 'Grey')
('"And Gus Taubs boy over in the tank plan."', 'Grey')
('"And Gus Taubs boy over in the tank plan."', 'Green')
('"And Gus Taubs boy over in the tank plan."', 'Green')
('"And Gus Taubs boy over in the tank plan."', 'Grey')
('"And Gus Taubs boy over in the tank plan."', 'Green')
('"And Russians shucks I cant  add em all up. I can’t even [Spring?] is against you Russian scene ?"', 'Yellow')
('"And Russians shucks I cant  add em all up. I can’t even [Spring?] is against you Russian scene ?"', 'Grey')
('"And Russians shucks I cant  add em all up. I can’t even [Spring?] is against you Russian scene ?"', 'Green')
('"And Russians shucks I cant  add em all up. I can’t even [Spring?] is against you Russian scene ?"', 'Grey')
('"And Russians shucks I cant  add em all up. I can’t even [Spring?] is against you Russian scene ?"', 'Grey')
('"Every time a hen lays an egg that eggs against you

In [122]:
#18.Get artifact titles, cultures, and media ranks where the period is not null
query = """
SELECT m.title, m.culture, me.rank
FROM artifact_metadata AS m
JOIN artifact_media AS me
    ON m.id = me.objectid
WHERE m.period IS NOT NULL
ORDER BY me.rank DESC;
"""

my_cursor.execute(query)
for row in my_cursor.fetchall():
    print(row)

('Globular Jar with Rounded Bottom and Flaring Lip', 'Korean', 140317)
('"Eye" Bead with Aqua and White Swirls', 'Chinese', 140315)
('Broad-Shouldered, "Hanging Jar" with Vertical Lip, Wide Mouth, and Landscape Décor', 'Chinese', 140314)
('Globular, Wide-Mouthed Jar with Two Loop Handles and Pale Lavender Splash', 'Chinese', 140313)
('Large Circular Bowl with Five Large Russet Splashes against a Ground of Russet Flecks', 'Chinese', 140310)
('Small Tea Bowl with Decoration of Moon, Clouds, and Blossoming Plum', 'Chinese', 140309)
('Small Water Coupe in the Form of a Spotted Frog', 'Chinese', 140308)
('Small Jade Sculpture in the Form of a Recumbent Ram', 'Chinese', 140307)
('Standing Lokapala Guardian Figure with Proper Right Arm Raised', 'Chinese', 140306)
('Medicine Buddha (Chinese, Yaoshi Rulai) Seated on a Lotus Pedestal and Holding a Small Medicine Jar in His Left Hand', 'Chinese', 140305)
('Plaque: Wild Goat and Deer', 'East Asian', 140273)
('Large Circular Bowl', 'Chinese', 13942

In [123]:
#19.Find artifact titles ranked in the top 10 that include the color hue "Grey".
query = """
SELECT m.title, me.rank, c.hue
FROM artifact_metadata AS m
JOIN artifact_media AS me ON m.id = me.objectid
JOIN artifact_colors AS c ON m.id = c.objectid
WHERE c.hue = 'Grey'
ORDER BY me.rank DESC
LIMIT 10;
"""

my_cursor.execute(query)
for row in my_cursor.fetchall():
    print(row)

('Portrait:  Jacopo Bassano', 270487, 'Grey')
('Portrait:  Jacopo Bassano', 270487, 'Grey')
('Portrait:  Jacopo Bassano', 270487, 'Grey')
('Portrait:  Jacopo Bassano', 270487, 'Grey')
('Portrait:  Jacopo Bassano', 270487, 'Grey')
('Portrait:  Jacopo Bassano', 270487, 'Grey')
('Portrait:  Jacopo Bassano', 270487, 'Grey')
('Portrait:  Jacopo Bassano', 270487, 'Grey')
('The Second Hour of Night', 268854, 'Grey')
('The Second Hour of Night', 268854, 'Grey')


In [124]:
#20.How many artifacts exist per classification, and what is the average media count
#for each?
query = """
SELECT 
    m.classification,
    COUNT(m.id) AS artifact_count,
    AVG(me.mediacount) AS average_media_count
FROM artifact_metadata AS m
JOIN artifact_media AS me 
    ON m.id = me.objectid
WHERE m.classification IS NOT NULL
GROUP BY m.classification
ORDER BY artifact_count DESC;
"""

my_cursor.execute(query)
for row in my_cursor.fetchall():
    print(row)

('Photographs', 1170, Decimal('0.0000'))
('Archival Material', 960, Decimal('0.0000'))
('Prints', 914, Decimal('0.0000'))
('Drawings', 626, Decimal('0.0000'))
('Coins', 390, Decimal('0.0000'))
("Artists' Tools", 343, Decimal('0.0000'))
('Fragments', 196, Decimal('0.0000'))
('Seals', 139, Decimal('0.0000'))
('Paintings', 61, Decimal('0.0000'))
('Sculpture', 48, Decimal('0.0000'))
('Vessels', 30, Decimal('0.0000'))
('Calligraphy', 20, Decimal('0.0000'))
('Jewelry', 18, Decimal('0.0000'))
('Textile Arts', 18, Decimal('0.0000'))
('Multiples', 10, Decimal('0.0000'))
('Albums', 8, Decimal('0.0000'))
('Gems', 8, Decimal('0.0000'))
('Audiovisual Works', 7, Decimal('0.0000'))
('Ritual Implements', 7, Decimal('0.0000'))
('Architectural Elements', 7, Decimal('0.0000'))
('Medals and Medallions', 4, Decimal('0.0000'))
('Amulets', 3, Decimal('0.0000'))
('Boxes', 2, Decimal('0.0000'))
('Tools and Equipment', 2, Decimal('0.0000'))
('Plaques', 2, Decimal('0.0000'))
('Manuscripts', 1, Decimal('0.0000'))